## Run this first if you want NDVI/NDWI basemaps

In [ ]:
import ee
ee.Authenticate()

True

## Vibes

In [ ]:
%load_ext autoreload
%autoreload 2

from geovibes.ui import GeoVibes

vibes = GeoVibes(
    start_date = '2024-01-01',
    end_date = '2025-01-01',
    enable_ee=False,
    verbose=True,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initializing GeoVibes...
  Found: /Users/christopherren/geovibes/faiss_db/new-mexico_dry_run_metadata.db
  Found: /Users/christopherren/geovibes/faiss_db/new-mexico_2024_google_metadata.db
  Found: /Users/christopherren/geovibes/faiss_db/new-mexico_metadata.db
Found 3 database(s) in /Users/christopherren/geovibes/faiss_db
📁 Found 3 databases in directory
💾 Connecting to local database: /Users/christopherren/geovibes/faiss_db/new-mexico_2024_google_metadata.db
✅ Database connection established successfully
🗺️  spatial extension loaded for geometry support
⚡ vss extension loaded for vector search
🔍 Detected embedding dimension: 64
📍 Using boundary file: /Users/christopherren/geovibes/geometries/new-mexico.geojson
⚠️  Earth Engine not available - S2/NDVI/NDWI basemaps skipped
Building UI...


HTML(value='\n        <style>\n        .widget-toggle-buttons button:nth-child(1).mod-active {\n            ba…

🗺️  Loading boundary layer: /Users/christopherren/geovibes/geometries/new-mexico.geojson
✅ Boundary layer added successfully


In [ ]:
Y
